In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
stock_name = "TATAPOWER.NS"

In [ ]:
import os

In [ ]:
model_dir = f"/content/drive/MyDrive/sp_models/{stock_name}"

if not os.path.exists(model_dir):
    os.makedirs(model_dir)


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [ ]:
data = yf.download("TATAPOWER.NS" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [ ]:
csv_file = os.path.join(model_dir, f"{stock_name}.csv")

In [ ]:
data.to_csv(csv_file)

In [ ]:
data.shape

(1322, 6)

In [ ]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-01,93.800003,100.000000,92.400002,97.150002,83.893021,33659894
2018-01-02,98.250000,100.250000,96.150002,99.699997,86.095039,29174233
2018-01-03,100.300003,101.599998,99.599998,100.800003,87.044937,23056832


Understanding Trends with in the Data

In [ ]:
# Sort the data points based on indexes just for confirmation 
data.sort_index(inplace = True)

In [ ]:
# Remove any duplicate index 
data = data.loc[~data.index.duplicated(keep='first')]

In [ ]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-05-08,201.50,203.199997,199.500000,202.699997,202.699997,7724617
2023-05-09,203.75,205.100006,202.399994,203.300003,203.300003,7956883
2023-05-10,203.50,204.350006,202.199997,204.149994,204.149994,3776084


In [ ]:
# Check for missing values 
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# Get the statistics of the data
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1322.000000,1322.000000,1322.000000,1322.000000,1322.000000,1.322000e+03
mean,118.647844,120.442057,116.548903,118.414751,115.130930,2.206170e+07
std,71.890963,72.815892,70.738208,71.712002,73.201211,2.925095e+07
min,27.250000,28.600000,27.000000,27.299999,25.914843,4.376310e+05
25%,63.400002,64.500000,61.787500,63.275001,59.669462,6.078360e+06
50%,80.750000,81.699997,79.724998,80.649998,72.565243,1.205366e+07
75%,205.225002,207.062496,202.337505,204.775002,204.775002,2.718178e+07
max,292.350006,298.049988,280.350006,289.799988,287.498932,3.606618e+08


In [ ]:
import plotly.graph_objects as go

# Check the trend in Closing Values 
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

Data Preparation

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [ ]:
# Filter only required data 
data = data[['Close' , 'Volume']]
data.head(3)

,Close,Volume
Date,,
2018-01-01,97.150002,33659894
2018-01-02,99.699997,29174233
2018-01-03,100.800003,23056832


In [ ]:
# Confirm the Testing Set length 
test_length = data[(data.index >= '2021-03-01')].shape[0]

In [ ]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [ ]:
X , Y = CreateFeatures_and_Targets(data , 32)

  0%|          | 0/1290 [00:00<?, ?it/s]

In [ ]:
# Check the shapes
X.shape , Y.shape

((1290, 32, 2), (1290,))

In [ ]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [ ]:
# Check Training Dataset Shape 
Xtrain.shape , Ytrain.shape

((748, 32, 2), (748,))

In [ ]:
# Check Testing Dataset Shape
Xtest.shape , Ytest.shape

((542, 32, 2), (542,))

In [ ]:
# Create a Scaler to Scale Vectors with Multiple Dimensions 
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:, :, i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [ ]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [ ]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [ ]:


checkpoint_path = os.path.join(model_dir, 'best_weights.h5')


In [ ]:
def save_object(obj , name : str):
    name_f = os.path.join(model_dir, f"{name}.pck")
    pickle_out = open(name_f,"wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    name_f = os.path.join(model_dir, f"{name}.pck")
    pickle_in = open(name_f,"rb")
    data = pickle.load(pickle_in)
    return data

In [ ]:
# Save your objects for future purposes 
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

Model Building

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=4, min_lr=0.00001,verbose = 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 2))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.15))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.15))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [ ]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [ ]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

In [ ]:
# Checking the model Structure 
model.summary()

In [ ]:
# Load the best weights
weight_path = checkpoint_path
model.load_weights(weight_path)

Visualize prediction on Test Set

In [ ]:
Predictions = model.predict(Xtest)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:
# Creating Sample Test Dataframe
test_dataframe_dict = {'Actual' : list(Actual) , 'Predicted' : list(Predictions)}
test_df = pd.DataFrame.from_dict(test_dataframe_dict)

test_df.index = data.index[-test_length:]

In [ ]:
test_df.head()

In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = test_df.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = test_df.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

Visualize Prediction on whole data

In [ ]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [ ]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [ ]:
Predictions = model.predict(Total_features)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:

fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

In [ ]:
y_true = Actual
y_pred = Predictions
loss = tf.keras.losses.mean_absolute_percentage_error(y_true, y_pred)
print(loss)

In [ ]:
#rmse
from math import sqrt
mse = tf.keras.losses.mean_squared_error(y_true, y_pred)
rmse = sqrt(mse)
print(mse, rmse)